In [1]:
import time
import pickle

import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

#from scipy.sparse import vstack 

from validation.data import *
from validation.scoring import bubbleup_score, BubbleUpMixin

from src.model import StarSpace

import torch
import torch.nn as nn
from torch import optim

/opt/conda/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print("Using device", device)

Using device cuda


In [3]:
SAMPLE_SIZE = 100000 #500000
SOC_LEVEL = 6
BUBBLE_UP = 3
PROD = False

In [4]:
MODEL = 'adversarial' #The folder to use

In [5]:
X_train, y_train = dot_train_data(SOC_LEVEL)
X_test, y_test, ids = indeed_test_data('../data/us/everything.csv', SAMPLE_SIZE, SOC_LEVEL, use_gcs=True)
if PROD == False:
    noprod_idx = get_soc_n(y_train.astype(str), 2) != 51
    X_train, y_train = X_train[noprod_idx], y_train[noprod_idx]

In [6]:
X_train = X_train.reset_index(drop=True)

In [7]:
# path = 'lmd-classify-dot/ss-models/bcohen/weights_e5'
# fs = GCSFileSystem(project='labor-market-data')
# with fs.open(path) as f:
#     embeddings = pickle.load(f)

with open('data/%s/weights_best_epoch' % MODEL, 'rb') as f:
    embeddings = pickle.load(f)

print(embeddings.shape)
embeddings = torch.FloatTensor(embeddings)
embeddings = nn.Embedding.from_pretrained(embeddings)

(12431, 100)


In [8]:
# path = 'lmd-classify-dot/ss-models/bcohen/train_vocab_e5'
# fs = GCSFileSystem(project='labor-market-data')
# with fs.open(path) as f:
#     vocab = pickle.load(f)

In [9]:
with open('data/%s/train_vocab_10000' % MODEL, 'rb') as f:
    vocab = pickle.load(f)

In [10]:
len(vocab)

12431

In [11]:
model = StarSpace(
    d_embed=100,
    vocabulary=vocab,
    k_neg = 10,
    input_embedding = embeddings)

In [12]:
X_train = model.get_positions(X_train)
X_test = model.get_positions(X_test)

for i in range(len(X_train)):
    for j in range(len(X_train[i])):
        X_train[i][j] = X_train[i][j].to(device)

for i in range(len(X_test)):
    for j in range(len(X_test[i])):
        X_test[i][j] = X_test[i][j].to(device)

In [13]:
X_train_emb = np.empty([X_train.shape[0],100])
X_test_emb = np.empty([X_test.shape[0],100])

In [14]:
start_time = time.time()
for i,doc in enumerate(X_train):
    X_train_emb[i] = model.embed_doc(torch.cat(doc)).cpu()
print(time.time() - start_time)

start_time = time.time()
for i,doc in enumerate(X_test):
    X_test_emb[i] = model.embed_doc(torch.cat(doc)).cpu()
print(time.time() - start_time)

3.8255770206451416
4.125118017196655


In [15]:
class BubbleUpLogisticRegression(BubbleUpMixin, LogisticRegression):
    pass

In [ ]:
Bubbles = BubbleUpLogisticRegression(C=2., solver='lbfgs', class_weight='balanced', 
                                     multi_class="multinomial", n_jobs=-1).set_bubbles(BUBBLE_UP)

Bubbles.fit(X_train_emb,y_train)

In [ ]:
y_pred = Bubbles.predict(X_test_emb)

In [ ]:
accuracy_score(get_soc_n(y_test.astype(str), BUBBLE_UP).astype(str), y_pred) 